In [12]:
!pip install -q transformers bitsandbytes accelerate torch tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 7.0 MB/s eta 0:00:00


In [20]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata

hf_token = userdata.get('COLAB_HF_TOKEN')

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

# Load the model pass HF secret token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

print("Model loaded successfully!")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully!


In [21]:
prompt_text = "The best part of waking up is"

inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

print("\nTokenized Input IDs:")
print(inputs['input_ids'])


with torch.no_grad():
  outputs = model(**inputs)

logits = outputs.logits

print("\nShape of the output logits:")
print(logits.shape)

last_token_logits = logits[0, -1, :] # Get logits for the last token in the sequence
predicted_token_id = torch.argmax(last_token_logits).item()

predicted_word = tokenizer.decode(predicted_token_id)

print(f"\nModel's next word prediction: '{predicted_word}'")



Tokenized Input IDs:
tensor([[   1,  415, 1489,  744,  302,  275, 1288,  582,  349]])

Shape of the output logits:
torch.Size([1, 9, 32000])

Model's next word prediction: 'coffee'
